# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [9]:
# import libraries
import re
import nltk
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.stem import WordNetLemmatizer
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score, fbeta_score, classification_report, precision_recall_fscore_support
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import hmean
from scipy.stats.mstats import gmean
import pandas as pd
import numpy as np
import os
import pickle
from sqlalchemy import create_engine
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [10]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterMessages2', con=engine)
X = df['message']
Y = df.iloc[:,4:]

In [11]:
# data sample for messages used in model as X. 
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [12]:
# data sample for 36 message classifications used in model as Y.
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [13]:
def tokenize(text):   
    # Replace all urls with a urlplaceholder string, extract them and replace urls with url placeholder string.
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    # Extract the word tokens from the provided text and use lemmanitizer method.
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    # List of clean tokens in messages.
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

In [15]:
pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ]))
            
        ])),

        ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
pipeline_fit = pipeline.fit(X_train, Y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [17]:
y_pred = pipeline_fit.predict(X_test)
y_pred_pd = pd.DataFrame(y_pred, columns = Y_test.columns)
for column in Y_test.columns:
    print('------------------------------------------------------\n')
    print('CATEGORY: {}\n'.format(column))
    print(classification_report(Y_test[column],y_pred_pd[column]))

------------------------------------------------------

CATEGORY: related

             precision    recall  f1-score   support

          0       0.69      0.34      0.46      1531
          1       0.83      0.95      0.89      5023

avg / total       0.80      0.81      0.79      6554

------------------------------------------------------

CATEGORY: request

             precision    recall  f1-score   support

          0       0.91      0.97      0.94      5423
          1       0.76      0.53      0.63      1131

avg / total       0.88      0.89      0.88      6554

------------------------------------------------------

CATEGORY: offer

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6524
          1       0.25      0.03      0.06        30

avg / total       0.99      1.00      0.99      6554

------------------------------------------------------

CATEGORY: aid_related

             precision    recall  f1-score   support


In [20]:
# Print classification report on test data
print(classification_report(Y_test.iloc[:,1:].values, np.array([x[1:] for x in y_pred]), target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.76      0.53      0.63      1131
               request       0.25      0.03      0.06        30
                 offer       0.76      0.58      0.66      2753
           aid_related       0.58      0.26      0.35       512
          medical_help       0.71      0.32      0.44       351
      medical_products       0.60      0.22      0.33       170
     search_and_rescue       0.24      0.05      0.09       115
              security       0.63      0.36      0.46       230
              military       0.00      0.00      0.00         0
           child_alone       0.72      0.67      0.70       438
                 water       0.82      0.70      0.75       755
                  food       0.75      0.54      0.63       592
               shelter       0.70      0.43      0.53        98
              clothing       0.56      0.34      0.42       140
                 money       0.48      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 35, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [21]:
# get aslo parameters to improve model
pipeline_fit.get_params()

{'memory': None, 'steps': [('features', FeatureUnion(n_jobs=1,
          transformer_list=[('text_pipeline', Pipeline(memory=None,
        steps=[('count_vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
    ...tfidf_transformer', TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True))]))],
          transformer_weights=None)),
  ('classifier',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=1.0, n_estimators=50, random_state=None),
              n_jobs=1))], 'features': FeatureUnion(n_jobs=1,
        transformer_list=[('text_pipeline', Pipeline(memory=None,
      steps=[('count_vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error

In [22]:
#custom estimator class is created with lessons learned in Case Study:Grid Search of Udacity
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [23]:
# new model with custom estimator
pipeline_new = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ])),            
            
            ('starting_verb', StartingVerbExtractor())
        ])),

        ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [24]:
parameters = {
    
        'features__text_pipeline__count_vectorizer__ngram_range': ((1, 1), (1, 2)),
        'classifier__estimator__learning_rate': [0.02, 0.04],
        'classifier__estimator__n_estimators': [20, 40] }

cv = GridSearchCV(pipeline_new, param_grid=parameters)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [25]:
# test score function
def test_score(y_test, y_pred):
    scores = pd.DataFrame(columns=['Category', 'F_score', 'Precision', 'Recall'])
    num = 0
    for cat in y_test.columns:
        precision, recall, f_score, support = precision_recall_fscore_support(y_test[cat], y_pred[:,num], average='weighted')
        scores.set_value(num+1, 'Category', cat)
        scores.set_value(num+1, 'F_score', f_score)
        scores.set_value(num+1, 'Precision', precision)
        scores.set_value(num+1, 'Recall', recall)
        num += 1
    print('Aggregated F_score:', scores['F_score'].mean())
    print('Aggregated Precision:', scores['Precision'].mean())
    print('Aggregated Recall:', scores['Recall'].mean())
    return scores

In [26]:
#testing scores of new model
cv.fit(X_train, Y_train)
Y_pred = cv.predict(X_test)
final_test_scores = test_score(Y_test, Y_pred)
final_test_scores

Aggregated F_score: 0.917448349101
Aggregated Precision: 0.92270663955
Aggregated Recall: 0.937777608246


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if __name__ == '__main__':
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: set_value is deprecated and will be removed in a future releas

,Category,F_score,Precision,Recall
1,related,0.665049,0.587372,0.766402
2,request,0.819335,0.861013,0.858102
3,offer,0.993139,0.990866,0.995423
4,aid_related,0.605178,0.708345,0.660208
5,medical_help,0.887006,0.928891,0.922948
6,medical_products,0.920703,0.922665,0.946445
7,search_and_rescue,0.962753,0.975314,0.974672
8,security,0.973758,0.965215,0.982453
9,military,0.947674,0.931045,0.964907
10,child_alone,1,1,1


In [27]:
# Print classification report on test data
print(classification_report(Y_test.iloc[:,1:].values, np.array([x[1:] for x in Y_pred]), target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.88      0.21      0.33      1131
               request       0.00      0.00      0.00        30
                 offer       0.81      0.25      0.38      2753
           aid_related       1.00      0.01      0.03       512
          medical_help       0.50      0.00      0.01       351
      medical_products       1.00      0.02      0.05       170
     search_and_rescue       0.00      0.00      0.00       115
              security       0.00      0.00      0.00       230
              military       0.00      0.00      0.00         0
           child_alone       0.82      0.49      0.61       438
                 water       0.78      0.69      0.73       755
                  food       0.83      0.32      0.46       592
               shelter       0.00      0.00      0.00        98
              clothing       1.00      0.01      0.01       140
                 money       0.50      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 35, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 9. Export your model as a pickle file

In [28]:
# Model is fine - tuned and Average overall accuracy is 92% and average f score is also approximately 92%
# Model is also exported as pickle file.
pickle.dump(cv, open('classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.